<a href="https://colab.research.google.com/github/Tiabet/Project_Market/blob/master/BertForSequence%2BAutoTokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.4 MB/s eta 0:00:00


In [ ]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")


('ratings_total.txt', <http.client.HTTPMessage at 0x7ec2a73ea500>)

In [ ]:
import pandas as pd

total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력


전체 리뷰 개수 : 200000


In [ ]:
total_data = total_data.drop(columns = ['ratings'])
total_data = total_data.sample(n=1000, random_state =42)

In [ ]:
total_data

,reviews
119737,마감이 안좋아요...실밥도 많고 바느질도 부족한 부분이 몇군데 있네요...교환받기 ...
72272,깨끗하게 잘 다듬어져 있어요. 맛도좋고요.
158154,재구매 배송빨라요 길냥이들이 잘먹어요~~ 대용량이라 좋네요~
65426,제품도 빨리 배송해주시고 꼼꼼하게 잘챙겨주셨어요
30074,기타 남 멋지고 예뻐요 여러 사은품도 좋아요
...,...
15354,빠배 벌레잘 잡음 단점 대형시켯는데도 크기가 생각보다작음 대형이아니라 중형크기임
8442,항상 여기서 사는데 s7엣지는 강화유리가 없어서 아쉽습니다
40287,색이 사진과는 다릅니다
78583,그냥 그래요 쏘쏘~~ 배송료 넘비싸요


In [ ]:
reviews = total_data['reviews']

In [ ]:
reviews

119737    마감이 안좋아요...실밥도 많고 바느질도 부족한 부분이 몇군데 있네요...교환받기 ...
72272                               깨끗하게 잘 다듬어져 있어요. 맛도좋고요.
158154                    재구매 배송빨라요 길냥이들이 잘먹어요~~ 대용량이라 좋네요~
65426                            제품도 빨리 배송해주시고 꼼꼼하게 잘챙겨주셨어요
30074                              기타 남 멋지고 예뻐요 여러 사은품도 좋아요
                                ...                        
15354          빠배 벌레잘 잡음 단점 대형시켯는데도 크기가 생각보다작음 대형이아니라 중형크기임
8442                       항상 여기서 사는데 s7엣지는 강화유리가 없어서 아쉽습니다
40287                                          색이 사진과는 다릅니다
78583                                  그냥 그래요 쏘쏘~~ 배송료 넘비싸요
37920     235길이는 폭은240인데 저에게 크네요ㅠ 235살껄 그런데 완전편함 검정도구입해야긋어요
Name: reviews, Length: 1000, dtype: object

In [ ]:
import torch
from transformers import AutoTokenizer, BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

pred_result = []
for review in reviews :
  inputs = tokenizer(review, return_tensors="pt")

  with torch.no_grad():
      logits = model(**inputs).logits
      #probabilities = torch.softmax(logits, dim=1) #가능성까지 확인하고 싶으면

  predicted_class_id = logits.argmax().item()
  pred_result.append(predicted_class_id)
  #print(predicted_class_id) #0이면 긍정, 1이면 부정
  #print(probabilities)
  #print(model.config.id2label[predicted_class_id])


In [ ]:
result = pd.DataFrame({'reviews' : reviews, 'Pos=0/Neg=1' : pred_result})

In [ ]:
result.reset_index(drop=True, inplace=True)

In [ ]:
result

,reviews,Pos=0/Neg=1
0,마감이 안좋아요...실밥도 많고 바느질도 부족한 부분이 몇군데 있네요...교환받기 ...,0
1,깨끗하게 잘 다듬어져 있어요. 맛도좋고요.,1
2,재구매 배송빨라요 길냥이들이 잘먹어요~~ 대용량이라 좋네요~,1
3,제품도 빨리 배송해주시고 꼼꼼하게 잘챙겨주셨어요,1
4,기타 남 멋지고 예뻐요 여러 사은품도 좋아요,1
...,...,...
995,빠배 벌레잘 잡음 단점 대형시켯는데도 크기가 생각보다작음 대형이아니라 중형크기임,0
996,항상 여기서 사는데 s7엣지는 강화유리가 없어서 아쉽습니다,0
997,색이 사진과는 다릅니다,0
998,그냥 그래요 쏘쏘~~ 배송료 넘비싸요,1


In [ ]:
result.to_csv('result.csv',index = False)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

inputs = tokenizer("저는 천재입니다", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits
    probabilities = torch.softmax(logits, dim=1)

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]